# Linear Combination of Unitaries

This notebook demonstrates how to represent and manipulate non-unitary
operations as quantum algorithms using the Linear Combination of
Unitaries (LCU) technique. We show how to apply a non-unitary matrix
to a quantum state using only unitary operations.

## Prerequisites

Before working through this notebook, you should be familiar with:

- Basic quantum computing concepts (qubits, unitary operators,
  measurements)
- Pauli operators (I, X, Y, Z) and their properties
- Quantum state preparation and measurement
- Python programming basics

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the Linear Combination of Unitaries (LCU) decomposition
   technique
2. Decompose a non-unitary operator into a linear combination of
   unitary operators (Pauli decomposition)
3. Implement LCU using Classiq's quantum programming framework
4. Analyze measurement results to verify the correct application of
   non-unitary operations

## Objective

The objective is to Apply the non-unitary matrix
$$
P =
\begin{pmatrix}
1 & 0\\
0 & 0
\end{pmatrix}
$$
on a 1-qubit quantum state and perform measurement on the output.

## Linear Combination of Unitaries (LCU)

The idea is to represent a non-unitary operator ($\hat{A}$) as a composition of unitary operators ($\hat{U}_i$'s):
$$
\hat{A} = \sum_{i=0}^{2^n-1} \alpha_i \hat{U}_i
$$
An example of such a decomposition is PauliTerms decomposition in which the set of $\{I, \hat{\sigma}_x, \hat{\sigma}_y, \hat{\sigma}_z \}$ (or the full set of their tensor products for operators acting on multiple qubits) is used as a basis for expansion and then the Frobenius inner product would help determining the unique expansion coefficients.

When decomposition coefficients $\alpha_i$'s are determined, one would prepare the following state on the auxilary registers :
$$
|\psi_0\rangle = \sum_i \sqrt{\frac{|\alpha_i|}{\lambda}}|i\rangle \Longleftrightarrow
\sqrt{\lambda} \langle i|\psi_0\rangle = \sqrt{|\alpha_i|}, \quad \lambda = \sum_i |\alpha_i|
$$
Since our objective is to find $\hat{A} |\psi\rangle$, in the next step, on would apply controlled-$\hat{U}_i$, $|\psi\rangle$ being the target and auxilary state $|i\rangle$ being the controller :
$$
\hat{A} |\psi\rangle = \sum_{i} \alpha_i \hat{U}_i |\psi\rangle = 
\sum_i \lambda |\langle\psi_0|i\rangle|^2 \hat{U}_i |\psi\rangle =
\lambda \langle\psi_0| . \left( \sum_i |i\rangle \hat{U}_i |\psi\rangle \langle i| \right) . |\psi_0\rangle
$$


## Implementation in Classiq

In [ ]:
# Execute this cell if you are not already authenticated
# and do not have a valid API token
import os
import classiq
# connecting through proxy
os.environ['http_proxy'] = "PROXY_ADDRESS" 
os.environ['https_proxy'] = "PROXY_ADDRESS"
classiq.authenticate()

Following is an LCU (Linear Combination of Unitaries) decomposition of the non-Unitary operator 

$$
P =
\begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
=
\frac{1}{2} \times 
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
+ 
\frac{1}{2} \times 
\begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
=
\frac{1}{2} \times 
\left(
I + \sigma_z
\right)
$$

In [1]:
import numpy as np
from classiq import *
from classiq.execution import ExecutionPreferences

NUM_QUBITS = 2
amps = [0.3, 0.7]
P = np.array([[1, 0],
              [0, 0]])


@qfunc
def lcu_controllers(controller: QNum, psi: QNum):
    # One can write P as 0.5 * ( I + Z)
    # This means :
    #    alpha_0 = alpha_1 = 0.5
    #    U_0 = I (controller == 0)
    #    U_1 = Z (controller == 1)
    #    
    control(ctrl=controller == 0, operand=lambda: apply_to_all(IDENTITY, psi))

    control(ctrl=controller == 1, operand=lambda: Z(psi))



@qfunc
def main(controller: Output[QNum], psi: Output[QNum]):
    # Defining the error bound and probability distribution
    error_bound = 0.01
    controller_probabilities = [0.5, 0.5]
    
    # Allocating the target and control qubits, respectively
    allocate(1, psi)
    allocate(1, controller)
    
    # Preparing psi
    RY(2 * np.arccos(np.sqrt(0.3)), psi)
    
    # Executing the Within-Apply function, the SELECT function 
    # is defined by lcu_controllers and the PREPARE function is 
    # defined by the inplace_prepare_state function.
    within_apply(
        compute=lambda: inplace_prepare_state(
            probabilities=controller_probabilities, bound=error_bound, target=controller
        ),
        action=lambda: lcu_controllers(controller, psi),
    )


quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)

To review the synthesized circuit in Classiq portal, execute :

## File Paths and Working Directory

This notebook assumes you are running from the `nonunitary_quantum_computing/`
directory. It will generate:

- `lcu-2x2.qmod` - Quantum model file (if `write_qmod` is executed)

This file is saved in the current working directory.

In [ ]:
show(quantum_program)

and to store the program in qmod, execute:

In [ ]:
write_qmod(quantum_model, "lcu-2x2")

## Summary

This notebook demonstrated the Linear Combination of Unitaries (LCU)
technique for implementing non-unitary operations on quantum computers.
Key takeaways include:

- Non-unitary operators can be decomposed into linear combinations of
  unitary operators (e.g., using Pauli decomposition)
- The LCU technique uses auxiliary qubits to control which unitary
  operation is applied
- Measurement results can be used to verify that the non-unitary
  operation was correctly implemented
- The decomposition $P = 0.5(I + Z)$ allows us to implement the
  projection operator $P$ using only unitary gates

## Further Reading

- [Classiq Documentation](https://docs.classiq.io/)
- Related notebooks: [Quantum Fourier Transform - Abelian groups
  case](../quantum_fourier_transform_abelian/)

Now we can execute the quantum program to get some results.

In [2]:
num_shots = 16000
Pxpsiamps = []
quantum_model_with_execution_preferences = \
set_execution_preferences(
    quantum_model,       
    ExecutionPreferences(
        num_shots=num_shots, 
        job_name=f"quantum primitives 2 - {num_shots} shots", 
        random_seed=767
    ),
)
quantum_program_with_execution_preferences = \
synthesize(
    quantum_model_with_execution_preferences
)
result = execute(quantum_program_with_execution_preferences).result()
Pxpsiamps.append(result[0].value.counts['00'] / num_shots)
Pxpsiamps.append(result[0].value.counts['11'] / num_shots)
    
print(f"number of shots : {num_shots} " + 
      f" |  P x psi (state 00 amplitude) : {Pxpsiamps[0]}" +
      f" |  P x psi (state 11 amplitude) : {Pxpsiamps[1]}")

number of shots : 16000  |  P x psi (state 00 amplitude) : 0.2995 |  P x psi (state 11 amplitude) : 0.7005


## Analysis

To find the result of $P|\psi\rangle$, one should perform measurements on the first qubit when the ancilla is in state $|0\rangle$. In other words, the objective would be to determine the amplitudes of composite states $|0\rangle \otimes |0\rangle$ and $|1\rangle \otimes |0\rangle$.

Checking our execution results, it is clear that the only composite states generated in the output are $|00\rangle$ and $|11\rangle$. This indictaes that :
$$ 
P |\psi\rangle = \sqrt{0.2995} |0\rangle
$$
which is consistent with the math :
$$
\begin{pmatrix}
1 & 0 \\ 
0 & 0
\end{pmatrix}
\times 
\left( \sqrt{0.3} |0\rangle + \sqrt{0.7} |1\rangle \right)
= \sqrt{0.3} |0\rangle
$$